In [1]:
from SAC.Normal_Policy import Normal_Policy
from SAC.Value_function import Value_function
from SAC.SoftActorCritc import SoftActorCritic, plot

import time
import gym
import numpy as np

import sys
sys.path.insert(0, './../../laser-hockey-env')
import laser_hockey_env as lh
from importlib import reload

import progressbar
from tensorboardX import SummaryWriter
import tensorflow as tf


In [2]:
env_name = "Laser_hockey"
env = lh.LaserHockeyEnv()

In [3]:
ac_space = env.action_space
o_space = env.observation_space
print(ac_space.low)
print(o_space)

[-1. -1. -1. -1. -1. -1.]
Box(16,)


In [4]:
ob=env.reset()

In [5]:
direct = "C:/Users/Maximilian/Documents/Studium/Informatik/3. Semester/Intelligent Systems/project/DeepRLHockey/code/SAC/weights/"
version = "v2_full_24"
save_path_new = direct+version

In [6]:
q_fct_config = {"hidden_layers": [256, 256]}
v_fct_config = {"hidden_layers": [256, 256]}
pi_fct_config = {"hidden_layers": [256, 256], "dim":3, "output_act_fct_mu":None}

SAC_agent = SoftActorCritic(o_space=o_space, 
                            a_space=ac_space, 
                            value_fct=Value_function, 
                            policy_fct=Normal_Policy, 
                            env= env,
                            discount=  0.93, 
                            tau = 0.05, 
                            batch_size=1024, 
                            alpha = 'auto',
                            lambda_V = 3e-4, 
                            lambda_Pi = 3e-4, 
                            lambda_Q = 3e-4,
                            dim_act = 3,
                            dim_obs = 16,
                            buffer_size = int(3e5),
                            q_fct_config=q_fct_config,
                            v_fct_config=v_fct_config,
                            pi_fct_config=pi_fct_config, 
                            save_path = save_path_new
                           )

In [7]:
player2 = lh.BasicOpponent()

In [8]:
writer = SummaryWriter('visualization/{}-{}'.format(env_name, version))

In [9]:
def eval_agent(n=100):
    win = 0
    draw = 0
    lose = 0
    for i in range(n):
        ob = env.reset()
    
        for _ in range(500):
            obs_agent2 = env.obs_agent_two()    
            a1 = SAC_agent.act_greedy( np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))[0]
            a2 = player2.act(obs_agent2)

            ob, r, d, info = env.step(np.hstack([a1,a2]))    
            if d: 
                win += (info["winner"] == 1)
                lose += (info["winner"] == -1)
                break
        draw += (info["winner"] == 0)
        env.close()
        
    return win/n, draw/n, lose/n
    

In [10]:

for i in range(1000):
    ob = env.reset()
    for _ in range(500):
        a = env.action_space.sample()
        ob_new, reward, done, _info = env.step(a) 
        reward += _info["reward_touch_puck"]
                
        SAC_agent.buffer.store(ob, a[:3], reward , ob_new, done)
                    
        ob = ob_new
        if done: break
for i in range(100):
    ob = env.reset()
    for _ in range(500):
        a = player2.act(ob)
        a2 = env.action_space.sample()[3:]
        ob_new, reward, done, _info = env.step(np.hstack([a,a2])) 
        reward += _info["reward_touch_puck"]
                
        SAC_agent.buffer.store(ob, a[:3], reward , ob_new, done)
                    
        ob = ob_new
        if done: break


In [11]:
def train(iter_fit=1000, max_steps= 1,  env_steps_max = 500, grad_steps = 1, burn_in =5, writer = None):
        start = SAC_agent._sess.run(SAC_agent.global_step)
        saver = SAC_agent._saver
        bar = progressbar.ProgressBar(max_value=iter_fit)
        # Init Statistics      
        winner_count = 0
        draw_count = 0
        total = start
        obs_dims = SAC_agent._config["dim_obs"]
        
        j = start
        for i in range(iter_fit):  
            ob = env.reset()     
            obs_agent2 = env.obs_agent_two()
            
            total_reward = 0
            
            for e_i in range(env_steps_max):#
                if j % 500 == 0:
                    env.render()
                        
                    
                if j < burn_in:
                    a = player2.act(ob)
                else:
                    a = SAC_agent.action(np.asarray(ob).reshape(1, obs_dims))
                    a = a[0]
                        
                a2 = player2.act(obs_agent2)                    
                comb_a = np.hstack([a,a2])
                
                (ob_new, reward, done, _info) = env.step(comb_a)
                
                ob_new_agent2 =env.obs_agent_two()
                    
                reward += _info["reward_touch_puck"]
                total_reward += reward
                    
                SAC_agent.buffer.store(ob, a, reward , ob_new, done)
                ob=ob_new
                obs_agent2 = ob_new_agent2
                
                if done:
                    break 
  
            if j  >= 0:
                for g_i in range(10):
                    update_value_target = False
                    if i % SAC_agent._config["target_update"] == 0:
                        update_value_target = True
                    loss_V_fct, loss_Q1_fct,loss_Q2_fct,loss_PI_fct= SAC_agent._train(update_value_target)
 
                

            j += 1
            if j > burn_in:
                winner_count +=  _info['winner'] == 1
                draw_count += _info['winner'] == 0
                total += 1    
                SAC_agent._sess.run(SAC_agent.increment_global_step_op)
                env.close()
                
            
            if j >burn_in and i % 250 == 0:
                saver.save(SAC_agent._sess, save_path_new)

            if writer and j> burn_in:
                if j%500 == 0: 
                    w, d, l = eval_agent(100)

                    writer.add_scalar('performance_evals/wins',w , total)
                    writer.add_scalar('performance_evals/draws',d , total)
                    writer.add_scalar('performance_evals/lose',l , total)
                writer.add_scalar('win_loss_ratio/wins',winner_count/total , total)
                writer.add_scalar('win_loss_ratio/draws',draw_count/total , total)
                writer.add_scalar('reward',total_reward , total)
            bar.update(i)   
        return winner_count

In [12]:
train(11419, burn_in = 0, writer = writer)

 19% (11419 of 60000) |###               | Elapsed Time: 4:34:04 ETA:  23:55:53

KeyboardInterrupt: 

In [13]:
SAC_agent._saver.save(SAC_agent._sess, save_path_new)

'C:/Users/Maximilian/Documents/Studium/Informatik/3. Semester/Intelligent Systems/project/DeepRLHockey/code/SAC/weights/v2_full_23'

In [ ]:
eval_agent(2000)

In [14]:

for i in range(10):
    ob = env.reset()
    for _ in range(1000):
        env.render()
        obs_agent2 = env.obs_agent_two()    
        #fddct = {SAC_agent.obs : np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"])}
        #a1 = SAC_agent._sess.run([SAC_agent._Policy.mu], feed_dict=fddct)[0][0]
        a1 = SAC_agent.act_greedy( np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))[0]
        a2 = player2.act(obs_agent2)
        
        ob, r, d, info = env.step(np.hstack([a1,a2]))    
        obs_agent2 = env.obs_agent_two()
        if d: break
env.close()

KeyboardInterrupt: 

In [15]:
env.close()